In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time

# Import API key
from Config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
temp = []
humidity= []
clouds = []
wind = []
lat = []
lng = []
missing_cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(lat_lng)

(-63.58333492984623, 76.10261018618723)


## Perform API Calls

In [3]:
# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key
for counter, city in enumerate(cities):
    if counter % 60 == 0 and counter != 0:
        print(f"sleeping for 60s after {counter} out of {len(cities)} requests")
        time.sleep(60)
    query = f"{url}&q={city}"
    response = requests.get(query).json()
    print(city)
    print("------------------")
    try:
        temp.append(response["main"]["temp"])
        humidity.append(response["main"]["humidity"])
        clouds.append(response["clouds"]["all"])
        wind.append(response["wind"]["speed"])
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
    except KeyError as e:
       
        print(str(response["cod"]) + str(e))
        print(response["message"])
        temp.append("nan")
        humidity.append("nan")
        clouds.append("nan")
        wind.append("nan")
        lat.append("nan")
        lng.append("nan")
        missing_cities.append(city)
        
        if response["cod"] == 429: 
            break

    

hobart
------------------
429'main'
Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription http://openweathermap.org/price


In [4]:
Weather_Dict = {
    "city": cities,
    "temp":temp,
    "humidity":humidity,
    "clouds":clouds,
    "Wind Speed":wind,
    "lat":lat,
    "lng":lng,

}

Weather_DF = pd.DataFrame(Weather_Dict)

Weather_DF.head()

ValueError: arrays must all be same length

In [ ]:
print(json.dumps(response, indent=4))